

# ANALISIS PERILAKU KONSUMEN MEGALINE

## Konten<a id='back'></a>

* [Pendahuluan](#intro)
* [Tahap 1. Inisialisasi data](#inisial)
* [Tahap 2. Mempersiapkan data](#prepare)
* [Tahap 3. Analisa Data](#analyze)
* [Tahap 4. Hipotesis](#hypothesis)
* [Kesimpulan Umum](#conclusion)
    * [Temuan](#end)
    * [Rekomendasi](#recom)

## Pendahuluan<a id='intro'></a>

Operator seluler Megaline merasa tidak puas karena sebagian besar pelanggan mereka masih menggunakan paket lama. Perusahaan tersebut ingin mengembangkan sebuah model yang bisa menganalisis perilaku konsumen dan merekomendasikan salah satu dari kedua paket terbaru Megaline: Smart atau Ultra.

Kamu punya akses untuk data perilaku para pelanggan yang sudah beralih ke paket terbaru (dari proyek kursus Analisis Data Statistik). Dalam tugas klasifikasi ini, kamu perlu mengembangkan sebuah model yang mampu memilih paket dengan tepat. Nah, karena kamu sudah menyelesaikan langkah pra-pemrosesan data, kamu bisa langsung lanjut ke tahapan pembuatan model.

Kembangkan sebuah model dengan tingkat accuracy yang setinggi-tingginya. Di proyek ini, ambang batas untuk tingkat accuracy adalah 0,75. Jangan lupa untuk memeriksa tingkat accuracy modelmu menggunakan test dataset, ya.

Instruksi proyek
1. Buka dan periksa file data secara teliti. File path: `/datasets/users_behavior.csv`.
2. Pisahkan data sumber menjadi `training set`, `validation set`, dan `test set`.
3. Periksa kualitas model yang berbeda dengan mengubah hyperparameter. 
4. Periksa kualitas model menggunakan test set.

Tugas tambahan: lakukan sanity check terhadap model. Data ini lebih kompleks dari data-data yang pernah dikerjakan sebelumnya. Jadi, ini memang bukan tugas yang mudah.

Deskripsi data
Setiap observasi dalam dataset yang kita miliki memuat informasi perilaku bulanan tentang satu pengguna. Informasi tersebut mencakup: 
* сalls — jumlah panggilan
* minutes — total durasi panggilan dalam satuan menit
* messages — jumlah pesan teks
* mb_used — traffic penggunaan internet dalam satuan MB
* is_ultimate — paket untuk bulan yang sedang berjalan (Ultimate - 1, Surf - 0)

## Inisialisasi Data<a id='inisial'></a>

memuat semua library yang diperlukan

In [37]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from joblib import dump
import joblib
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression


## Mempersiapkan Data<a id='prepare'></a>

In [38]:
# Mengunduh dan memeriksa data
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [39]:
#memeriksa informasi umum
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


terlihat bahwa data yang dimiliki terdiri dari kolom `calls`, `minutes`, `messages`, `mb_used`, dan `is_ultra`. data teridiri dari 3214 data, dan tidak ada nilai yang hilang. akan tetapi kolom `calls` dan `messages` memiliki tipe data yang salah sehingga perlu diperbaiki.

In [40]:
#memeriksa duplicate
data.duplicated().sum()

0

terlihat bahwa tidak ada data yang terduplikasi

In [41]:
# mengubah tipe data kolom calls menjadi integer
data['calls'] = data['calls'].astype('int')

# mengubah tipe data kolom messages menjadi integer
data['messages'] = data['messages'].astype('int')

# mengecek tipe data yang baru
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


sekarang terlihat bahwa `calls` dan `messages` sudah berganti tipe data menjadi int

In [42]:
#memuat informasi umum tambahan
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


terlihat bahwa data yang dimiliki terdiri dari kolom `calls`, `minutes`, `messages`, dan `mb_used` merupakan data kuantitatif yang sangat bervariasi sedangkan `is_ultra` hanya berisi data biner `0` dan `1`.

## Membuat model machine learning<a id='machine'></a>

Membuat model machine learning yang terbaik diperlukan beberapa tahapan diantaranya: pemisahan dataset, memilih algoritma, menguji algoritma, tuning hyperparamater.

### Memisahkan dataset

Pisahkan data sumber menjadi `training set`, `validation set`, dan `test set`.
Disini kita akan menentukan paket apa yang direkomendasikan sebagai target berdasarkan beberapa fitur yang berkaitan. Fitur-fitur tersebut diantaranya `calls`, `minutes`, `messages` dan `mb_used`.

Karena kita hanya memilki satu buah dataset, kita akan membaginya kedalam beberapa kelompok untuk membuat model machine learning. Pembagian dipecah menjadi 60% dataset untuk training, 20% dataset untuk validasi dan 20% untuk testing.

In [43]:
# membagi dataset menjadi 60% training, 20% validasi dan 20% test

# membagi dataset megaline menjadi 60% untuk training dan 40% untuk (validasi + test)
df_train, df_temp = train_test_split(data, test_size=0.6, random_state=147)

# membagi dataset temporary menjadi 50% untuk validation dan 50% untuk test (dari 40% data asli)
df_validation, df_test = train_test_split(df_temp, test_size=0.5, random_state=147)

In [44]:
# memisahkan kolom is_ultra dari dataset
# membagi dataset training menjadi fitur dan target
features_train = df_train.drop(['is_ultra'],axis=1)
target_train = df_train['is_ultra']

# membagi dataset validation menjadi fitur dan target
features_valid = df_validation.drop(['is_ultra'],axis=1)
target_valid = df_validation['is_ultra']

# membagi dataset test menjadi fitur dan target
features_test = df_test.drop(['is_ultra'],axis=1)
target_test = df_test['is_ultra']

# menampilkan shape dari training dan validation
print('features_train:',features_train.shape)
print('target_train:',target_train.shape,'\n')

print('features_valid:',features_valid.shape)
print('target_valid:',target_valid.shape,'\n')

print('features_test:',features_test.shape)
print('target_test:',target_test.shape)

features_train: (1285, 4)
target_train: (1285,) 

features_valid: (964, 4)
target_valid: (964,) 

features_test: (965, 4)
target_test: (965,)


dari `features_train` terlihat bahwa diperlukan 1285 data untuk dijadikan `training set`, yang berarti sekitar 60% dari jumlah data yang ada. sementara `validation set` diperlukan sekitar 964 data dan `features_test` terlihat bahwa diperlukan 965 data.

## Memeriksa kualitas model<a id='quality'></a>

### Periksa kualitas model yang berbeda dengan mengubah hyperparameter. 

In [45]:
# Membuat temporary variabel untuk tuning hyperparameters
best_result = 0
best_depth = 0

# Pengujian kedalaman model decision tree (depth -> 1 ~ 15)
for depth in range(1, 15):
    # Membuat model decision tree
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # Melatih model menggunakan features dan target train
    model.fit(features_train, target_train)
    # Menghitung akurasi menggunakan features dan target validation
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

# Menampilkan hasil tuning hyperparameters
print("Akurasi Model Terbaik (Decision Tree)")
print(f"Akurasi\t: {best_result:.5f}")
print(f"Depth\t: {best_depth}")

Akurasi Model Terbaik (Decision Tree)
Akurasi	: 0.79876
Depth	: 3


Berdasarkan hasil di atas terlihat kualitas nilai yang dimasukkan untuk `validation test`, terlihat bahwa nilai akurasi tertinggi adalah pada kedalaman `3` yaitu sekitar `0.798`.

### Periksa Kualitas Model Menggunakan Test Set

In [46]:
# Evaluasi model terbaik pada test set
prediction_test = best_model.predict(features_test)
result_test = accuracy_score(target_test, prediction_test)

# Evaluasi model terbaik pada training set
prediction_train = best_model.predict(features_train)
result_train = accuracy_score(target_train, prediction_train)

# Menampilkan hasil evaluasi pada training set dan test set
print('Akurasi Akhir')
print('Training set:', result_train)
print('Test set:', result_test)

Akurasi Akhir
Training set: 0.7937743190661478
Test set: 0.7989637305699482


terlihat bahwa akurasi akhir yang di dapatkan dengan `training set` adalah `0.793` sedangkan menggunakan `test set` adalah `0.798`

In [47]:
# Mencari accuracy model terbaik
best_score = 0
best_est = 0
for est in range(1, 15):
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

final_model = RandomForestClassifier(random_state=54321, n_estimators=10)
final_model.fit(features_train, target_train)

# menampilkan output        
print("Akurasi Model Terbaik (Random Forest)")
print(f"Akurasi\t\t: {best_score:.5f}")
print(f"Depth\t\t: {best_depth}")
print(f"N_Estimators\t: {best_est}")

Akurasi Model Terbaik (Random Forest)
Akurasi		: 0.79357
Depth		: 3
N_Estimators	: 14


akurasi model `random forest` adalah `0.793` dengan `depth` 3 dan `n_estimator` 14

## Sanity Check<a id='sanity'></a>

In [48]:
# Sanity Check dengan menggunakan mean squarred error
best_model = None
best_result = 1
best_est = 0
best_depth = 0
for est in range(10, 30, 50):
    for depth in range (1, 11):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        prediction_valid = model.predict(features_valid)
        result = mean_squared_error(target_valid, prediction_valid)**0.5
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("RMSE dari model terbaik pada validation set:", best_result)
print("n_estimators:", best_est)
print("best_depth:", depth)


RMSE dari model terbaik pada validation set: 0.3863799176844318
n_estimators: 10
best_depth: 10


terlihat bahwa RMSE validation set adalah `0.386` dengan `n_estimators` 10 dan `best_depth` 10

In [49]:
model = LinearRegression()
model.fit(features_train, target_train)
prediction_valid = model.predict(features_valid)

result = mean_squared_error(target_valid, prediction_valid)**0.5
print("RMSE dari model regresi linier pada validation set:", result)

RMSE dari model regresi linier pada validation set: 0.44428262267721674


RSME dari model regresi linier pada validation set adalah senilai `0.444`. Nilai RMSE pada model linier berbeda (lebih besar) dengan nilai random forest regression sehingga kemungkinan model ini overfitting.

In [50]:
model = LogisticRegression(random_state=54321, solver='liblinear') 
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train) 
score_valid = model.score(features_valid, target_valid) 

print("Accuracy model regresi logistik berdasarkan training set:", score_train)
print("Accuracy model regresi logistik berdasarkan validation set:", score_valid)

Accuracy model regresi logistik berdasarkan training set: 0.7167315175097276
Accuracy model regresi logistik berdasarkan validation set: 0.6939834024896265


Skor akurasi pada training set lebih tinggi dari akurasi pada validation. Hal ini sesuai dengan sebelumnya mengindikasikan adanya adanya overfitting.

In [51]:
# Final model
# melatih model menggunakan dataset training dan validasi untuk mendapatkan hasil yang lebih akurat menggunakan random forest

# menggabungkan dataset training dan validasi
merge_df = pd.concat([df_train,df_validation],axis=0)
print('New Dataset:',merge_df.shape)

# membagi dataset akhir menjadi fitur dan target
features = merge_df.drop(['is_ultra'],axis=1)
target = merge_df['is_ultra']

# max_depth = 3
# n_estimators = 14
best_model = RandomForestClassifier(random_state=12345, n_estimators=14, max_depth=3)

# melatih model terbaik
best_model.fit(features,target)

New Dataset: (2249, 5)


RandomForestClassifier(max_depth=3, n_estimators=14, random_state=12345)

In [52]:
# uji kelayakan model (sanity check)
accuracy = best_model.score(features_test,target_test)

# menampilkan output
print("Akurasi best_model:",accuracy)

Akurasi best_model: 0.7958549222797927


Bisa dilihat ternyata akurasinya masih berada disekitar 79% dan masih diatas ambang batas akurasi skor 75%.

## Kesimpulan<a id='conclusion'></a>

1. jumlah training set 60% adalah `1285` data, validation set 20% adalah `964` data, dan testing set `965` data.
2. Berdasarkan hasil di atas terlihat kualitas nilai yang dimasukkan untuk `validation test`, terlihat bahwa nilai akurasi tertinggi adalah pada kedalaman 3 yaitu sekitar `0.798` menggunakan `decision tree`.
3. akurasi akhir yang di dapatkan dengan `training set` adalah `0.793` sedangkan menggunakan `test set` adalah `0.798`
4. akurasi model `random forest` adalah `0.793` dengan `depth` 3 dan `n_estimator` 14.
5. terlihat bahwa RMSE validation set adalah `0.386` dengan `n_estimators` 10 dan `best_depth` 10.
6. Bisa dilihat ternyata akurasinya `sanity check` masih berada disekitar 79% dan masih diatas ambang batas akurasi skor 75%.

### Temuan<a id='end'></a>

1. memisahkan data set menjadi 3 bagian, `training set`, `validation set`, dan `testing set` dengan persentase 60:20:20
2. RSME dari model regresi linier pada validation set adalah senilai `0.444`. Nilai RMSE pada model linier berbeda (lebih besar) dengan nilai random forest regression sehingga kemungkinan model ini overfitting.

### Rekomendasi<a id='recom'></a>

Berdasarkan hasil analisis model, rekomendasi yang diberikan adalah untuk menggunakan model yang dikembangkan dalam memilih paket untuk pelanggan. Karena model memiliki akurasi yang baik dan konsisten, pelanggan yang diidentifikasi melalui model ini kemungkinan besar akan mendapatkan paket yang paling sesuai dengan kebutuhan mereka.

Untuk lebih spesifik:
* Pelanggan yang diprediksi oleh model untuk beralih ke paket "Ultra" sebaiknya ditawari paket "Ultra".
* Pelanggan yang diprediksi oleh model untuk beralih ke paket "Smart" sebaiknya ditawari paket "Smart".

Keputusan ini didukung oleh analisis akurasi dan keandalan model dalam memprediksi pilihan paket yang tepat berdasarkan data perilaku pelanggan. Menggunakan pendekatan ini akan membantu Megaline dalam meningkatkan kepuasan pelanggan dengan menawarkan paket yang lebih sesuai dengan kebutuhan individu pelanggan.

[Kembali](#back)